In [ ]:
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt

get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('config', "InlineBackend.figure_format ='retina'")

In [ ]:
ev = pd.read_csv('~/TX_appleseed_lenders_public/data/evictions_historic_merged.csv')


In [ ]:
ev['precinct'] = ev['court_id'].apply(lambda x: int(x/10))


In [ ]:
ev2 = ev[['precinct','GEOID_2020','judgment_amt','NAME_2020']].copy(deep=True)


In [ ]:
hex_code = ['#F9C80E','#FF1053','#354CA1','#CC0035','#59C3C3']
ev2['hex_code'] = ev2['precinct'].apply(lambda x: hex_code[x-1])

In [ ]:
ev2['judgment_amt'].mean()


In [ ]:
ev2[ev2['precinct']==1]['judgment_amt'].mean()


In [ ]:
ev2[ev2['GEOID_2020']==48113016610]['judgment_amt'].mean()


In [ ]:
plot = ev2[ev2['GEOID_2020']==48113016610][['judgment_amt']].plot(kind='bar',figsize=(10,10),legend=True)
plot.set_title('Judgement Amounts in 1 tract', fontsize=24)

plt.savefig('../results/judgmentbar.png', dpi=1200)

In [ ]:
ev3 = ev2[ev2['GEOID_2020']==48113016610]

In [ ]:
ev2['County Mean'] = ev2['judgment_amt'].mean()
ev2['Precinct Mean'] = ev2.apply(lambda x: ev2[ev2['precinct']==x['precinct']]['judgment_amt'].mean(),axis=1)
ev2['Tract Mean'] = ev2.apply(lambda x: ev2[ev2['GEOID_2020']==x['GEOID_2020']]['judgment_amt'].mean(),axis=1)


In [ ]:
ev4 = ev2[ev2['GEOID_2020']==48113016610][['County Mean','Precinct Mean','Tract Mean']].groupby(by='Tract Mean').mean().reset_index()


In [ ]:
plot2 = ev4.plot(kind='bar',figsize=(10,10),legend=True)
plot2.set_xticklabels(['48113016610'],rotation=0);
plot2.set_title('Tract, County, and Precinct Means for 1 GEOID', fontsize=24)

plt.savefig('../results/geoidmeansbar.png', dpi=1200)

In [ ]:
ev2['County Mean'] = ev2['judgment_amt'].mean()
ev2['Precinct Mean'] = ev2.apply(lambda x: ev2[ev2['precinct']==x['precinct']]['judgment_amt'].mean(),axis=1)
ev2['Tract Mean'] = ev2.apply(lambda x: ev2[ev2['GEOID_2020']==x['GEOID_2020']]['judgment_amt'].mean(),axis=1)
ev2['judgment_count'] = ev2.apply(lambda x: len(ev2[ev2['GEOID_2020']==x['GEOID_2020']]),axis=1)
new_df = ev2[['County Mean','Precinct Mean','Tract Mean','GEOID_2020','hex_code','judgment_count','NAME_2020']]

In [ ]:
def plot_tracts(data,t,verbose=False):
    data2 = data[data['GEOID_2020']==t][['Tract Mean','Precinct Mean','County Mean']].groupby(by='Tract Mean').mean().reset_index()
    jc = data[data['GEOID_2020']==t]['judgment_count'].max()
    c = data[data['GEOID_2020']==t]['hex_code'].iloc[0]
    n = data[data['GEOID_2020']==t]['NAME_2020'].iloc[0]
    plot = data2.plot(kind='bar',figsize=(10,10),legend=True,color=['k',c,'gray'])
    plot.set_title('Comparative Mean Eviction Judgments by Hierarchy for: \n {} with {} Judgments'.format(n,jc),fontsize=15)
    plot.set_xticklabels([str(t)],rotation=0);
    plot.set_xlabel("Tract   ---   Precinct   ---  County",fontsize=12)
    plot.set_ylabel("Judgment ($)",fontsize=12)
    plt.ylim([0,4000])
    if verbose:
        plot.figure.savefig('../results/judgment_avg_{}.pdf'.format(t))
        plt.close(plot.figure)
        return 'Saved Figure as results/judgment_avg_{}.pdf'.format(t)
    return plot

In [ ]:

def plot_tracts_ratio(data,t,verbose=False):
    data2 = data[data['GEOID_2020']==t][['Tract Mean','Precinct Mean','County Mean']].groupby(by='Tract Mean').mean().reset_index()
    data2 = data2/data2['County Mean'][0] * 100
    jc = data[data['GEOID_2020']==t]['judgment_count'].max()
    c = data[data['GEOID_2020']==t]['hex_code'].iloc[0]
    n = data[data['GEOID_2020']==t]['NAME_2020'].iloc[0]
    plot = data2.plot(kind='bar',figsize=(10,10),legend=True,color=['k',c,'gray'],hatch=['/'])
    plot.set_title('Ratio of Comparative Mean Eviction Judgments by Hierarchy for: \n {} with {} Judgments'.format(n,jc),fontsize=15)
    plot.set_xticklabels([str(t)],rotation=0);
    plot.set_xlabel("Tract   ---   Precinct   ---  County",fontsize=12)
    plot.set_ylabel("Judgments as Percent of County Mean",fontsize=12)
    plt.ylim([0,200])
    if verbose:
        plot.figure.savefig('../results/ratio_judgment_avg_{}.pdf'.format(t))
        plt.close(plot.figure)
        return 'Saved Figure as results/ratio_judgment_avg_{}.pdf'.format(t)
    return plot

In [ ]:
plot_tracts(new_df,48113016610,True)


In [ ]:
plot_tracts_ratio(ev2,48113019214,True)
